In [1]:
import os
import json
import pandas as pd

#Read one or many files into a pandas dataframe
class DataFrameBuilder(object):
    def __init__(self):
        self.data_frame = None
        self._data_list = []
    def read_to_pandas(self, path):
        with open(path) as data_file:
            if (path.endswith(".csv")):
                data = pd.read_csv(data_file)
            elif path.endswith(".json"):
                data = pd.read_json(data_file)
            else:
                raise ValueError("No parser exists for parsing %s" + path)
            self._data_list.append(data)
        self.data_frame = pd.concat(self._data_list)

#List all the files available in the selected immuta data source
def list_files(data_frame_builder, startpath):
    for root, dirs, files in os.walk(startpath):
        for file in files:
            path = os.path.join(root, file)
            print path
            #optionally build the pandas dataframe from the files
            data_frame_builder.read_to_pandas(path)

### This is your immuta root directory where all your data sources will appear
If you have none, subscribe to some from the Immuta storefront, otherwise, append a data source of interest to the path.  This will read **ALL** your files to the DF, so be careful what you point it to

### Do something below with your [pandas dfb.data_frame](http://pandas.pydata.org/pandas-docs/stable/10min.html)

### Here are all the data sources applicable for training the course recommendation engine.  They come from 3 catalogies.

## Training Record data - STEP report
- Tara Accessible STEP Report

## Training catalog dataset
- Combined Course Catalog (combined catalog for August, October and December version)
- Training Update Catalog (catalog for only the latest December version)
- Course Description Catalog (extended course description data for all courses in the combined catalog)

## Social platform Plazza dataset
- Tara Accessible Users Info Plazza
- TARA Accessible User Info HRI
- Tara Accessible User Relation 
- Tara Accessible Social Group  
- Tara Accessible Group Member  
- Tara Accessible Project  
- Blog Project
- Blog Social Group

## Here are a few points to explain how these tables are linked to each other
- "Tara Accessible STEP Report" table is the most important table that link together other two datasets
- "tara_accessible_master_id" column is the unified user id that links all the tables together
- "Tara Accessible STEP Report" table has the 'course id for catalog matching' column to match to 'Course#' column on both Combined Course Catalog and Training Update Catalog table
- "Course Description Catalog" table is used for fuzzy match.  Its "coursedes" column has detailed course description which can be used for fuzzy match to determine similarity of the courses. 
- All the "Tara Accessible XXX" tables have a "tara_accessible_master_id" column to reference each row to a user  
- "Tara Accessible User Relation" table has two columns asscociated with users because each row represents a relationship between two users.  The two columns indicate user relation are - "tara_accessible_master_id" & "tara_accessible_related_master_id"
- Table Blog Project & table Blog Social Group don't have a user id column, but they can be linked back to table Tara Accessible Project & table Tara Accessible Social Group respectively.  Blog Project table's containerid column is linked to Tara Accessible Project's parentobjectid column and Blog Social Group table's containerid column is linked to Tara Accessible Social Group's groupid column
![alt text](socialPlazzasSchema.png "Social Plazza Dataset Schema")


In [2]:
# Here are the file paths to local all the tables

# load Training catalog dataset
combinedCourseCatalog = pd.read_csv('immuta/Combined Course Catalog/Combined Course Catalog.csv')  # combined catalog for Aug, Oct and Dec version
trainingUpdateCatalog = pd.read_csv('immuta/Training Update Catalog/Training Update Catalog.csv')  # catalog for only Dec version
combinedDescription = pd.read_csv('immuta/Course Description Catalog/Course Description Catalog.csv')  # catalog for only Dec version
# Load Training Record data - STEP report
taraAccessibleStepReport = pd.read_csv('immuta/Tara Accessible STEP Report/Tara Accessible STEP Report.csv')  
# Load Social platform Plazza data
taraAccessibleUsersInfoPlazza = pd.read_csv('immuta/Tara Accessible Users Info Plazza/Tara Accessible Users Info Plazza.csv')  
taraAccessibleUserInfoHRI = pd.read_csv('immuta/TARA Accessible User Info HRI/TARA Accessible User Info HRI.csv')  
taraAccessibleUserRelation = pd.read_csv('immuta/Tara Accessible User Relation/Tara Accessible User Relation.csv')  
taraAccessibleSocialGroup = pd.read_csv('immuta/Tara Accessible Social Group/Tara Accessible Social Group.csv')  
taraAccessibleGroupMember = pd.read_csv('immuta/Tara Accessible Group Member/Tara Accessible Group Member.csv')  
taraAccessibleProject = pd.read_csv('immuta/Tara Accessible Project/Tara Accessible Project.csv')  
blogProject = pd.read_csv('immuta/Blog Project/Blog Project.csv')  
blogSocialGroup = pd.read_csv('immuta/Blog Social Group/Blog Social Group.csv')

### Below are the demonstration on how all those data sources are connected

tara_accessible_master_id column is the unified user id that can link all the STEP data source and all the soical plazza data sources except Blog Project and Blog Social Group. 

In [3]:
print('###########################################')
print('Catalog Dataset')
print('###########################################')

print("matching Tara Accessible STEP Report to Combined Course Catalog using 'course id for catalog matching' column on Tara Accessible STEP Report and 'Course#' column on Combined Course Catalog")
StepReportMatchingCombinedCatalog = pd.merge(taraAccessibleStepReport, combinedCourseCatalog, how='inner', left_on='course id for catalog matching', right_on='Course#')
print("Number of rows matched: ", StepReportMatchingCombinedCatalog.shape[0])

print("matching Tara Accessible STEP Report to Training Update Catalog using 'course id for catalog matching' column on Tara Accessible STEP Report and 'Course#' column on Training Update Catalog")
StepReportMatchingTrainingUpdateCatalog = pd.merge(taraAccessibleStepReport, trainingUpdateCatalog, how='inner', left_on='course id for catalog matching', right_on='Course#')
print("Number of rows matched: ", StepReportMatchingTrainingUpdateCatalog.shape[0])

print("matching Course Description Catalog to Combined Course Catalog using 'course url' column on Course Description Catalog and 'Course Url' column on Combined Course Catalog")
CombinedDescriptionMatchingCombinedCatalog = pd.merge(combinedDescription, combinedCourseCatalog, how='inner', left_on='course url', right_on='Course Url')
print("Number of rows matched: ", CombinedDescriptionMatchingCombinedCatalog.shape[0])

print('###########################################')
print('Social Plazza Dataset')
print('###########################################')

print("matching Tara Accessible STEP Report to Tara Accessible Users Info Plazza using tara_accessible_master_id column on both table")
StepReportMatchingUserInfoPlazza = pd.merge(taraAccessibleStepReport, taraAccessibleUsersInfoPlazza, how='inner', on='tara_accessible_master_id')
print("Number of rows matched: ", StepReportMatchingUserInfoPlazza.shape[0])

print("matching Tara Accessible Users Info Plazza to TARA Accessible User Info HRI using tara_accessible_master_id column on both table")
UserInfoPlazzaMatchingUserInfoHRI = pd.merge(taraAccessibleUsersInfoPlazza, taraAccessibleUserInfoHRI, how='inner', on='tara_accessible_master_id')
print("Number of rows matched: ", UserInfoPlazzaMatchingUserInfoHRI.shape[0])

print("matching Tara Accessible Users Info Plazza to Tara Accessible User Relation using tara_accessible_master_id column on both table")
UserInfoPlazzaMatchingUserRelation = pd.merge(taraAccessibleUsersInfoPlazza, taraAccessibleUserRelation, how='inner', on='tara_accessible_master_id')
print("Number of rows matched: ", UserInfoPlazzaMatchingUserRelation.shape[0])

print("matching Tara Accessible Users Info Plazza to Tara Accessible Social Group using tara_accessible_master_id column on both table")
UserInfoPlazzaMatchingSocialGroup = pd.merge(taraAccessibleUsersInfoPlazza, taraAccessibleSocialGroup, how='inner', on='tara_accessible_master_id')
print("Number of rows matched: ", UserInfoPlazzaMatchingSocialGroup.shape[0])

print("matching Tara Accessible Users Info Plazza to Tara Accessible Group Member using tara_accessible_master_id column on both table")
UserInfoPlazzaMatchingGroupMember = pd.merge(taraAccessibleUsersInfoPlazza, taraAccessibleGroupMember, how='inner', on='tara_accessible_master_id')
print("Number of rows matched: ", UserInfoPlazzaMatchingGroupMember.shape[0])

print("matching Tara Accessible Users Info Plazza to Tara Accessible Project using tara_accessible_master_id column on both table")
UserInfoPlazzaMatchingProject = pd.merge(taraAccessibleUsersInfoPlazza, taraAccessibleProject, how='inner', on='tara_accessible_master_id')
print("Number of rows matched: ", UserInfoPlazzaMatchingProject.shape[0])

print("matching Tara Accessible Social Group to Blog Social Group")
taraAccessibleSocialGroupMatchBlogSocialGroup = pd.merge(taraAccessibleSocialGroup, blogSocialGroup, how='inner', left_on='groupid', right_on='containerid')
print("Number of rows matched: ", taraAccessibleSocialGroupMatchBlogSocialGroup.shape[0])

print("matching Tara Accessible Social Group to Tara Accessible Project")
taraAccessibleSocialGroupMatchTaraAccessibleProject = pd.merge(taraAccessibleSocialGroup, taraAccessibleProject, how='inner', left_on='groupid', right_on='parentobjectid')
print("Number of rows matched: ", taraAccessibleSocialGroupMatchTaraAccessibleProject.shape[0])

print("matching Tara Accessible Project to Blog Project")
taraAccessibleProjectMatchBlogProject = pd.merge(taraAccessibleProject, blogProject, how='inner', left_on='parentobjectid', right_on='containerid')
print("Number of rows matched: ", taraAccessibleProjectMatchBlogProject.shape[0])

print("matching Tara Accessible Social Group to Blog Project")
taraAccessibleSocialGroupMatchBlogProject = pd.merge(taraAccessibleSocialGroup, blogProject, how='inner', left_on='groupid', right_on='containerid')
print("Number of rows matched: ", taraAccessibleSocialGroupMatchBlogProject.shape[0])

###########################################
Catalog Dataset
###########################################
matching Tara Accessible STEP Report to Combined Course Catalog using 'course id for catalog matching' column on Tara Accessible STEP Report and 'Course#' column on Combined Course Catalog
('Number of rows matched: ', 133558)
matching Tara Accessible STEP Report to Training Update Catalog using 'course id for catalog matching' column on Tara Accessible STEP Report and 'Course#' column on Training Update Catalog
('Number of rows matched: ', 55924)
matching Course Description Catalog to Combined Course Catalog using 'course url' column on Course Description Catalog and 'Course Url' column on Combined Course Catalog
('Number of rows matched: ', 11234)
###########################################
Social Plazza Dataset
###########################################
matching Tara Accessible STEP Report to Tara Accessible Users Info Plazza using tara_accessible_master_id column on both table
('

### Below are showing all the columns for all tables

In [4]:
# print out datafrome information for all data sources
print('=== Training catalog dataset ===')
combinedCourseCatalog.info()
trainingUpdateCatalog.info()
combinedDescription.info()
print('=== Training Record data ===')
taraAccessibleStepReport.info()
print('=== Social platform Plazza data ===')
taraAccessibleUsersInfoPlazza.info()
taraAccessibleUserInfoHRI.info()
taraAccessibleUserRelation.info()
taraAccessibleSocialGroup.info()
taraAccessibleGroupMember.info()
taraAccessibleProject.info()
blogProject.info()
blogSocialGroup.info()

=== Training catalog dataset ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80213 entries, 0 to 80212
Data columns (total 13 columns):
id                          80213 non-null object
Language                    80213 non-null object
Solution Area               80213 non-null object
Curriculum                  80213 non-null object
Series                      80213 non-null object
Course Title                80213 non-null object
Course#                     79072 non-null object
Course Url                  80201 non-null object
Asset Type                  79072 non-null object
Estimated Duration Hours    78971 non-null object
Skillport                   79072 non-null object
CD                          78978 non-null object
Replaces                    76137 non-null object
dtypes: object(13)
memory usage: 8.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69723 entries, 0 to 69722
Data columns (total 14 columns):
Language                    69723 non-null object
Solution